In [1]:
import gradio as gr
from diffusers import DDPMScheduler, UNet2DModel, DDPMPipeline
from accelerate import Accelerator
import os
import torch

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["NVIDIA_VISIBLE_DEVICES"] = "0"

In [3]:
output_dir="model_230402"

In [ ]:
model=UNet2DModel.from_pretrained("./{}/unet/".format(output_dir)).to(torch.device("cuda"))

In [ ]:
noise_scheduler = DDPMScheduler(num_train_timesteps=1000)

In [ ]:
mixed_precision='fp16'
gradient_accumulation_steps=1

In [ ]:
accelerator = Accelerator(
    mixed_precision=mixed_precision,
    gradient_accumulation_steps=gradient_accumulation_steps, 
    log_with="tensorboard",
    project_dir=os.path.join(output_dir, "logs")
)

In [ ]:
pipeline = DDPMPipeline(unet=accelerator.unwrap_model(model), scheduler=noise_scheduler)

In [ ]:
def return_images(seed):
    images = pipeline(batch_size = 1,
                      generator=torch.manual_seed(seed)).images
    return images[0]                 

In [ ]:
# def greet(name):
#     return "Hello " + name + "!"

In [ ]:
demo = gr.Interface(
    fn=return_images,
    inputs=gr.Slider(0, 100,label="goodliness"),
#     inputs=None,
#     inputs=gr.Textbox(lines=2, placeholder="Name Here..."),
    outputs="image",
)
demo.launch()